In [12]:
import pandas as pd
from fuzzywuzzy import fuzz
from dateutil.parser import parse
import tkinter as tk
from tkinter import filedialog, Tk, StringVar, Label, OptionMenu

def load_files():
    """ Load Files and Data based on User Input """
    
    # Create a Tkinter root window
    root = tk.Tk()
    root.withdraw()

    print("Select the first CSV file you would like to compare.")
    file1 = filedialog.askopenfilename(title="Select CSV file 1", filetypes=[("CSV Files", "*.csv")])
    print("Select the second CSV file you would like to compare.")
    file2 = filedialog.askopenfilename(title="Select CSV file 2", filetypes=[("CSV Files", "*.csv")])
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    
    # Retrieve columns and send to list variables
    file1_columns = df1.columns.tolist()
    file2_columns = df2.columns.tolist()
    
    # Print the columns of each file for the user to see
    print("Columns in first file:")
    for i, col in enumerate(file1_columns):
        print(f"{i}. {col}")
    col1 = int(input("Enter the index of the column containing the name or ID of the first file: "))
    
    print("Columns in second file:")
    for i, col in enumerate(file2_columns):
        print(f"{i}. {col}")
    col2 = int(input("Enter the index of the column containing the name or ID of the second file: "))
    
    print("Columns in first file:")
    for i, col in enumerate(file1_columns):
        print(f"{i}. {col}")
    col3 = int(input("Enter the index of the date column in file 1 you would like to compare against file 2: "))
    
    print("Columns in second file:")
    for i, col in enumerate(file2_columns):
        print(f"{i}. {col}")
    col4 = int(input("Enter the index of the date column in file 2 you would like to compare: "))
    
    # Check that the indices are valid
    while True:
        try:
            df1[df1.columns[col3]].apply(parse)
            df2[df2.columns[col4]].apply(parse)
            break
        except ValueError:
            print("The columns you entered do not contain dates. Please enter columns that contain dates.")
            col3 = int(input("Enter the index of the date column in file 1 you would like to compare against file 2: "))
            col4 = int(input("Enter the index of the date column in file 2 you would like to compare: "))
            
        except IndexError:
            print("The index you entered is invalid. Please enter a valid index.")
            col1 = int(input("Enter the index of the column containing the name or ID of the first file: "))
            col2 = int(input("Enter the index of the column containing the name or ID of the second file: "))
            col3 = int(input("Enter the index of the date column in file 1 you would like to compare against file 2: "))
            col4 = int(input("Enter the index of the date column in file 2 you would like to compare: "))
    
    return df1, df2, file1_columns[col1], file2_columns[col2], file1_columns[col3], file2_columns[col4]

df1, df2, col1, col2, col3, col4 = load_files()

# Find the best match between the two rows
for index1, row1 in df1.iterrows():
    best_match_score = 0
    best_match_row = None

    # Find the best match with any row in df2
    for index2, row2 in df2.iterrows():
        score = fuzz.ratio(str(row1[col1]), str(row2[col2]))
        if score > best_match_score:
            best_match_score = score
            best_match_row = row2

    # Print out the contents of the best match
    if best_match_score > 0:
        print('Best match for row {} in file1.csv:'.format(index1))
        print('Name/ID: ' + row1[col1])
        print(col3 + ':', row1[col3])
        print('Best match in file2.csv:')
        print('Name/ID: ' + best_match_row[col2])
        print(col4 + ':', best_match_row[col4])
        if parse(row1[col3]) == parse(best_match_row[col4]):
            print('Values of {} and {} match.\n'.format(col3, col4))
        else:
            print('Values of {} and {} do not match.\n'.format(col3, col4))
            new_date_str = input('Enter the correct date for {} in the format "DD-Mon-YYYY": '.format(col1))

            # Convert the input to a datetime object
            new_date = parse(new_date_str)

            # Update the dates in both sheets
            df1.at[index1, col3] = new_date.strftime('%d-%b-%Y')
            df2.at[best_match_row.name, col4] = new_date.strftime('%d-%b-%Y')
            print('{} updated to: {}'.format(col3, new_date_str))
            
            if parse(row1[col3]) == parse(best_match_row[col4]):
                print('Values updated. Values of {} and {} now match.\n'.format(col1, col2))
            else:
                print('Values still do not match')
                break
    else:
        print('No match found for row {} in file1.csv.'.format(index1))
        
df1.to_csv('ExampleC3_updated.csv', index=False)
df2.to_csv('ExampleC3_AGOL_updated.csv', index=False)

Select the first CSV file you would like to compare.
Select the second CSV file you would like to compare.
Columns in first file:
0. Project Name
1. Final C3 Date


Enter the index of the column containing the name or ID of the first file:  0


Columns in second file:
0. Project Description
1. Final C3 Construction


Enter the index of the column containing the name or ID of the second file:  0


Columns in first file:
0. Project Name
1. Final C3 Date


...

Columns in second file:
0. Project Description
1. Final C3 Construction


Enter the index of the column in file 2 you would like to compare:  1


Best match for row 0 in file1.csv:
Name/ID: Big One
Final C3 Date: 30-Jan-08
Best match in file2.csv:
Name/ID: Big One LLC
Final C3 Construction: 30, January 2008
Values of Final C3 Date and Final C3 Construction match.

Best match for row 1 in file1.csv:
Name/ID: Small guy / ross
Final C3 Date: 24-Feb-18
Best match in file2.csv:
Name/ID: Ross
Final C3 Construction: 24, march 1999
Values of Final C3 Date and Final C3 Construction do not match.



Enter the correct date for Project Name in the format "DD-Mon-YYYY":  24-FEb-18


Final C3 Date updated to: 24-FEb-18
Values updated. Values of Project Name and Project Description now match.

Best match for row 2 in file1.csv:
Name/ID: 1874 budda hand
Final C3 Date: 17, January 1999
Best match in file2.csv:
Name/ID: 1874 budda hand St
Final C3 Construction: 17, January 1999
Values of Final C3 Date and Final C3 Construction match.

